# Iterative

Реализовать итеративные версии mergesort и quicksort.

Тесты продолжаем писать.

# Import

In [1]:
import os

while os.getcwd().split("/")[-1] != "algorithms_python":
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
import random
import time
import numpy as np
from typing import Any, Callable, List, Optional

# Timer

time.perf_counter() vs time.process_time()

https://stackoverflow.com/questions/52222002/what-is-the-difference-between-time-perf-counter-and-time-process-time

In [3]:
def timer(func: Callable) -> Callable:
    def wrapper(*args, **kwargs) -> Any:
        start_time = time.perf_counter()  # можно через time.process_time()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()  # можно через time.process_time()
        execution_time = end_time - start_time
        print(f"Execution time of {func.__name__}: {execution_time:.4f} secs")
        return result

    return wrapper

# Merge sort

![merge sort](../imgs/merge_sort.png)

![comparison](../imgs/comparison.png)

In [6]:
def merge(arr: List[int | float], left: int, mid: int, right: int):
    # Вычисляем размеры двух подмассивов для слияния
    n1 = mid - left + 1
    n2 = right - mid
    
    arr1 = arr[left:left + n1]
    arr2 = arr[mid + 1:mid + 1 + n2]
    

    i = 0    # индекс для левого подмассива (arr1)
    j = 0    # индекс для правого подмассива (arr2)
    k = left # индекс для основного массива (arr)
    
    # Сливаем временные списки обратно в основной массив
    while i < n1 and j < n2:
        if arr1[i] <= arr2[j]:
            arr[k] = arr1[i]
            i += 1
        else:
            arr[k] = arr2[j]
            j += 1
        k += 1
    
    while i < n1:
        arr[k] = arr1[i]
        i += 1
        k += 1
    
    while j < n2:
        arr[k] = arr2[j]
        j += 1
        k += 1

@timer
def mergesort_iter(arr: List[int | float]) -> List[int | float]:
    arr = arr.copy()
    n = len(arr)
    
    currSize = 1 
    while currSize <= n - 1:
        
        # Выбираем начальные точки различных подмассивов текущего размера
        leftStart = 0
        while leftStart < n - 1:
            
            mid = min(leftStart + currSize - 1, n - 1)
            rightEnd = min(leftStart + 2 * currSize - 1, n - 1)
            
            merge(arr, leftStart, mid, rightEnd)
            
            leftStart += 2 * currSize

        currSize = 2 * currSize
        
    return arr

# Quick sort

![quick sort](../imgs/quick_sort.png)

In [16]:
def partition(arr: List[int | float], low, high):
    """
    Функция разделения массива относительно опорного элемента.
    Размещает элементы меньше опорного слева, больше - справа.

    Args:
        arr: массив для сортировки
        low: начальный индекс (low)
        high: конечный индекс (high)

    Returns:
        индекс опорного элемента после разделения
    """
    i = low - 1  # индекс меньшего элемента (изначально -1)
    x = arr[high]  # опорный элемент (pivot) - выбираем последний элемент

    for j in range(low, high):
        if arr[j] <= x:
            i = i + 1
            arr[i], arr[j] = arr[j], arr[i]

    # Помещаем опорный элемент на правильную позицию
    arr[i + 1], arr[high] = arr[high], arr[i + 1]
    return i + 1  # возвращаем индекс опорного элемента


@timer
def quicksort_iter(arr: List[int | float]) -> List[int | float]:
    """
    Итеративная реализация быстрой сортировки с использованием стека.
    Эмулирует рекурсию с помощью явного стека.

    Args:
        arr: массив для сортировки
    """
    arr = arr.copy()
    low = 0
    high = len(arr) - 1

    # Создаем вспомогательный стек для хранения границ подмассивов
    stack = [0] * (len(arr))  # стек для хранения границ [low, high]

    # Инициализируем вершину стека
    top = -1

    # Помещаем начальные значения low и high в стек
    top = top + 1
    stack[top] = low
    top = top + 1
    stack[top] = high

    # Продолжаем извлекать из стека, пока он не пуст
    while top >= 0:

        # Извлекаем high и low из стека (в обратном порядке)
        high = stack[top]
        top = top - 1
        low = stack[top]
        top = top - 1

        p = partition(arr, low, high)

        # Если есть элементы слева от опорного,
        # то помещаем левую часть в стек
        if p - 1 > low:
            top = top + 1
            stack[top] = low
            top = top + 1
            stack[top] = p - 1

        # Если есть элементы справа от опорного,
        # то помещаем правую часть в стек
        if p + 1 < high:
            top = top + 1
            stack[top] = p + 1
            top = top + 1
            stack[top] = high
    return arr

# Tests

In [35]:
random_list = list(np.random.randint(
    low=np.iinfo(np.int64).min, high=np.iinfo(np.int64).max, size=10_000_000
))

result_merge_sort_random_list = mergesort_iter(random_list)
result_quick_sort_random_list = quicksort_iter(random_list)

Execution time of mergesort_iter: 21.0384 secs
Execution time of quicksort_iter: 16.4930 secs


In [37]:
result_quick_sort_random_list

[np.int64(-9223370247842365083),
 np.int64(-9223368366757316486),
 np.int64(-9223367951897123049),
 np.int64(-9223358709679413523),
 np.int64(-9223354895676898404),
 np.int64(-9223354164296974765),
 np.int64(-9223345784804494142),
 np.int64(-9223341292948410068),
 np.int64(-9223341202640844417),
 np.int64(-9223340591733593793),
 np.int64(-9223340190277190984),
 np.int64(-9223339483203494980),
 np.int64(-9223339325055024520),
 np.int64(-9223336202645806927),
 np.int64(-9223335125975058408),
 np.int64(-9223330914372690089),
 np.int64(-9223330579741561013),
 np.int64(-9223326267121551017),
 np.int64(-9223324518247303634),
 np.int64(-9223324300457659446),
 np.int64(-9223320919922558382),
 np.int64(-9223319285721491133),
 np.int64(-9223318618722255392),
 np.int64(-9223316261028344661),
 np.int64(-9223315399335681481),
 np.int64(-9223314806563823513),
 np.int64(-9223314110410427242),
 np.int64(-9223313926950010175),
 np.int64(-9223312075062069134),
 np.int64(-9223311590477292659),
 np.int64(

In [38]:
sorted(random_list)

[np.int64(-9223370247842365083),
 np.int64(-9223368366757316486),
 np.int64(-9223367951897123049),
 np.int64(-9223358709679413523),
 np.int64(-9223354895676898404),
 np.int64(-9223354164296974765),
 np.int64(-9223345784804494142),
 np.int64(-9223341292948410068),
 np.int64(-9223341202640844417),
 np.int64(-9223340591733593793),
 np.int64(-9223340190277190984),
 np.int64(-9223339483203494980),
 np.int64(-9223339325055024520),
 np.int64(-9223336202645806927),
 np.int64(-9223335125975058408),
 np.int64(-9223330914372690089),
 np.int64(-9223330579741561013),
 np.int64(-9223326267121551017),
 np.int64(-9223324518247303634),
 np.int64(-9223324300457659446),
 np.int64(-9223320919922558382),
 np.int64(-9223319285721491133),
 np.int64(-9223318618722255392),
 np.int64(-9223316261028344661),
 np.int64(-9223315399335681481),
 np.int64(-9223314806563823513),
 np.int64(-9223314110410427242),
 np.int64(-9223313926950010175),
 np.int64(-9223312075062069134),
 np.int64(-9223311590477292659),
 np.int64(

In [40]:
result_merge_sort_random_list == sorted(random_list)
# result_quick_sort_random_list == sorted(random_list)

True

In [22]:
random_w_dublicates_list = np.random.randint(
    low=-10, high=10, size=10_000
)

result_merge_sort_random_w_dublicates_list = mergesort_iter(random_w_dublicates_list)
result_quick_sort_random_w_dublicates_list = quicksort_iter(random_w_dublicates_list)

Execution time of mergesort_iter: 0.0393 secs
Execution time of quicksort_iter: 0.5320 secs


In [24]:
all_same_list = [5] * 10_000

result_merge_sort_all_same_list = mergesort_iter(all_same_list)
result_quick_sort_all_same_list = quicksort_iter(all_same_list)

Execution time of mergesort_iter: 0.0086 secs
Execution time of quicksort_iter: 1.8160 secs


In [25]:
sorted_list = list(range(10_000))

result_merge_sort_sorted_list = mergesort_iter(sorted_list)
result_quick_sort_sorted_list = quicksort_iter(sorted_list)

Execution time of mergesort_iter: 0.0133 secs
Execution time of quicksort_iter: 1.9296 secs


In [26]:
reversed_sorted = list(range(10_000, 0, -1))

result_merge_sort_reversed_sorted = mergesort_iter(reversed_sorted)
result_quick_sort_reversed_sorted = quicksort_iter(reversed_sorted)

Execution time of mergesort_iter: 0.0174 secs
Execution time of quicksort_iter: 1.2841 secs
